In [100]:
%matplotlib inline
import proj1_helpers as utils
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload
import algorithms as ML_alg
import preprocessing_functions as prf

(y, tx, event_ids) = utils.load_csv_data("../train.csv")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[[  1.38470000e+02   5.16550000e+01   9.78270000e+01   2.79800000e+01
    9.10000000e-01   1.24711000e+02   2.66600000e+00   3.06400000e+00
    4.19280000e+01   1.97760000e+02   1.58200000e+00   1.39600000e+00
    2.00000000e-01   3.26380000e+01   1.01700000e+00   3.81000000e-01
    5.16260000e+01   2.27300000e+00  -2.41400000e+00   1.68240000e+01
   -2.77000000e-01   2.58733000e+02   2.00000000e+00   6.74350000e+01
    2.15000000e+00   4.44000000e-01   4.60620000e+01   1.24000000e+00
   -2.47500000e+00   1.13497000e+02]
 [  1.60937000e+02   6.87680000e+01   1.03235000e+02   4.81460000e+01
   -9.99000000e+02  -9.99000000e+02  -9.99000000e+02   3.47300000e+00
    2.07800000e+00   1.25157000e+02   8.79000000e-01   1.41400000e+00
   -9.99000000e+02   4.20140000e+01   2.03900000e+00  -3.01100000e+00
    3.69180000e+01   5.01000000e-01   1.03000000e-01   4.47040000e+01
   -1.91600000e+00   1.64546000e+02

In [2]:
(tx_clean, y_clean) = prf.get_clean_data(tx,y)

y_clean_bin = prf.pass_data_to_zero_one(y_clean).reshape([y_clean.shape[0],1])


std_tx_clean = prf.standardize(tx_clean)

In [3]:
std_tx_clean.shape

(68114, 30)

In [81]:
from matplotlib.mlab import PCA
pca = PCA(tx_clean)
pca.Y.shape

final_tx = ML_alg.build_poly(x = prf.standardize(pca.Y), degree=3)
final_tx.shape

(68114, 91)

In [115]:
initial_w = np.zeros([91,1])

final_tx = ML_alg.build_poly(x = std_tx_clean, degree=3)


(w, loss ) = ML_alg.gradient_descent(tx = final_tx, y = y_clean_bin, initial_w = initial_w,cost = 'reg_logistic',
                        lambda_= 0.1, gamma = 1e-7, update_gamma= False, max_iters = 10000)

GD(9999/9999), cost: reg_logistic: loss=28736.201756802995, w=[  1.37401624e-01   8.12258250e-01  -3.28149127e-01   4.40151544e-01
   5.25522272e-01   1.35953297e-01   5.85826293e-02  -1.60993884e-01
   3.18085279e-01  -2.65912393e-01   9.69327802e-02  -2.51944690e-01
   4.21563158e-01   2.65823014e-01   4.82625666e-01   3.23006178e-03
   2.90652828e-03   2.72200812e-01  -1.85489316e-02  -3.90675865e-03
   5.00002436e-01   3.19546111e-02  -2.08134114e-01  -1.02592613e-01
  -6.80534576e-02  -1.90942193e-02  -1.41002807e-02   1.51379029e-01
   1.18790118e-02  -1.20640584e-02  -8.80713588e-02  -1.07810225e+00
  -1.34368521e-01  -5.94751127e-01   1.87526231e-01   2.69894526e-01
   1.70332532e-01   6.68027187e-02  -6.93092950e-01   1.25087815e-01
  -5.67395036e-02  -4.46616415e-02   1.64124455e-01   1.22931824e-01
  -4.06039317e-02  -1.18575097e-01   3.84517143e-02   2.74172546e-02
  -1.37851573e-01   4.50837767e-02   3.20302412e-02   1.85773807e-02
   7.23331746e-02   4.98194114e-02  -5.90

In [116]:
def evaluate(tx,y,w):
    true = 0
    false = 0
    for i in range(y.shape[0]):
        res = tx[i].dot(w.reshape([91,1]))
        if(abs(1-res) < abs(res)):
            if (y[i] == 1):
                true += 1
            else:
                false += 1
        if(abs(1-res) > abs(res)):
            if (y[i] == 0):
                true += 1
            else:
                false += 1
    return (true,false)

In [117]:
(true, false) = evaluate(final_tx, y_clean_bin, w)

print(true / (true + false))

0.8038288751211204


In [98]:
def put_nan_to_0(tx):
    tx_nan_to_zero = np.zeros(tx.shape)
    for i in range(tx.shape[0]):
        for j in range(tx.shape[1]):
            if(tx[i,j] != 0):
                tx_nan_to_zero = tx[i,j]
    return tx_nan_to_zero
                      
def put_nan_to_mean(tx, y):
    tx_nan_to_mean = tx
    (tx_clean, y_clean) = prf.get_clean_data(tx, y)
    means = tx_clean.mean(axis=0)
    for i in range(tx.shape[0]):
        for j in range(tx.shape[1]):
            if(tx[i,j] == -999.0):
                print("a")
                tx_nan_to_mean[i,j] = means[j]
    return tx_nan_to_mean
                      


In [112]:
std_tx = ML_alg.build_poly(prf.standardize(put_nan_to_mean(tx,y)),degree=3)

std_tx.mean()

y_bin = prf.pass_data_to_zero_one(y)

[  1.22004529e+02   4.92398193e+01   8.11819816e+01   5.78959617e+01
   2.42538280e+00   3.76321358e+02  -8.63803906e-01   2.37309984e+00
   1.89173324e+01   1.58432217e+02   1.43760943e+00  -1.28304708e-01
   4.62224384e-01   3.87074191e+01  -1.09730480e-02  -8.17107200e-03
   4.66602072e+01  -1.95074680e-02   4.35429640e-02   4.17172345e+01
  -1.01191920e-02   2.09797178e+02   9.79176000e-01   9.36375742e+01
  -4.28021117e-03  -1.20614822e-02   5.78469592e+01  -1.26155629e-02
  -3.85375080e-03   7.30645914e+01]


In [114]:
(w, loss ) = ML_alg.gradient_descent(tx = std_tx, y = y_bin, initial_w = initial_w,cost = 'reg_logistic',
                        lambda_= 0.5, gamma = 1e-7, update_gamma= False, max_iters = 10000)

MemoryError: 

In [113]:
(true, false) = evaluate(std_tx, y_bin, w)

print(true / (true + false))

0.76214
